In [188]:
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import warnings
from time import time
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')

In [51]:
TRAIN_DIR = 'training_set\\training_set\\data'
TEST_DIR = 'test_set\\test_set\\data'

In [52]:
IMG_SIZE = 32

In [53]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]


def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [54]:
train_data = create_train_data()

100%|█████████████████████████████████████████████████████████████████████████████| 8005/8005 [00:14<00:00, 563.29it/s]


In [62]:
x = np.array([i[0] for i in train_data])
y = [i[1] for i in train_data]

In [63]:
y = np.array([i[0] for i in y])

In [57]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [58]:
test_data = process_test_data()

100%|██████████████████████████████████████████████████████████████████████████████| 2023/2023 [00:25<00:00, 80.87it/s]


In [97]:
x_test = np.array([i[0] for i in test_data])
y_test = [i[1] for i in test_data]

In [65]:
y_test = np.array([i[0] for i in y_test])

In [72]:
x.shape, y.shape, x_test.shape, y_test.shape

((1024, 8005), (1, 8005), (1024, 2023), (1, 2023))

In [71]:
x = x.reshape((x.shape[1]*x.shape[2],x.shape[0]))
x_test = x_test.reshape((x_test.shape[1]*x_test.shape[2],x_test.shape[0]))
y = y.reshape((1,y.shape[0]))
y_test = y_test.reshape((1,y_test.shape[0]))

In [73]:
x = x/255
x_test = x_test/255

In [118]:
x_train = x[:, :5000]
y_train = y[:, :5000]

x_dev = x[:, 5000:]
y_dev = y[:, 5000:]

In [155]:
n_x, n_y = x_train.shape[0], y_train.shape[0]

In [203]:
layer_dim = [n_x, 200, 100, 50, n_y]

In [204]:
def sigmoid(Z):
    
    A = 1/(1 + np.exp(-Z))
    
    return A

In [205]:
def relu(Z):

    A = np.maximum(0,Z)

    return A

In [206]:
def d_relu(dA, Z):
    
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0 ] = 0
    
    return dZ

In [207]:
def init_parameters(layer_dimensions):
    
    L = len(layer_dimensions) - 1
    parameters = {}
    
    for l in range(L):
        parameters['W' + str(l + 1)] = np.random.randn(layer_dimensions[l + 1], layer_dimensions[l]) * np.sqrt(2/layer_dimensions[l])
        parameters['b' + str(l + 1)] = np.zeros((layer_dimensions[l + 1], 1))
        
    return parameters    

In [208]:
def init_adam(parameters):
    
    L = len(parameters) // 2
    v = {}
    s = {}
    for l in range(L):
        v['dW' + str(l + 1)] = np.zeros((parameters['W' + str(l + 1)].shape[0], parameters['W' + str(l + 1)].shape[1]))
        v['db' + str(l + 1)] = np.zeros((parameters['b' + str(l + 1)].shape[0], parameters['b' + str(l + 1)].shape[1]))
        #s["dW" + str(l + 1)] = np.zeros((parameters["W" + str(l + 1)].shape[0], parameters["W" + str(l + 1)].shape[1]))
        #s["db" + str(l + 1)] = np.zeros((parameters["b" + str(l + 1)].shape[0], parameters["b" + str(l + 1)].shape[1]))
        
    return v

In [209]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    b4 = parameters['b4']
    
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = relu(Z3)
    Z4 = np.dot(W4, A3) + b4
    A4 = sigmoid(Z4)
    
    cache = {'Z1':Z1, 'Z2':Z2, 'Z3':Z3, 'Z4':Z4,
            'A1':A1, 'A2':A2, 'A3':A3, 'A4':A4}
    
    return A4, cache

In [210]:
def compute_cost(Y, A4, parameters, lambd):
    
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    W4 = parameters["W4"]
    
    regularized_term = (lambd/(2*m)) * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)) + np.sum(np.square(W4)))

    cost = (-1/m) * np.sum((Y * np.log(A4)) + ((1-Y) * np.log(1-A4))) + regularized_term
    
    return cost

In [211]:
def backward_propagation(X, Y, cache, parameters, lambd):
   
    m = Y.shape[1]
    Z1 = cache['Z1']
    Z2 = cache['Z2'] 
    Z3 = cache['Z3']
    Z4 = cache['Z4']
    A1 = cache['A1']
    A2 = cache['A2']
    A3 = cache['A3']
    A4 = cache['A4']
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    
    
    dZ4 = A4 - Y
    dW4 = (1/m) * np.dot(dZ4, A3.T) + (lambd * W4/m)
    db4 = (1/m) * np.sum(dZ4, axis = 1, keepdims=True)
    
    dA3 = np.dot(W4.T, dZ4)
    dZ3 = d_relu(dA3, Z3)
    dW3 = (1/m) * np.dot(dZ3, A2.T) + (lambd * W3/m)
    db3 = (1/m) * np.sum(dZ3, axis = 1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = d_relu(dA2, Z2)
    dW2 = (1/m) * np.dot(dZ2, A1.T) + (lambd * W2/m)
    db2 = (1/m) * np.sum(dZ2, axis = 1, keepdims=True) 
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = d_relu(dA1, Z1)
    dW1 = (1/m) * np.dot(dZ1, X.T) + (lambd * W1/m)
    db1 = (1/m) * np.sum(dZ1, axis = 1, keepdims=True)    
    
    grads = {'dW1' : dW1, 'dW2' : dW2, 'dW3' : dW3, 'dW4': dW4, 'db1' : db1, 'db2' : db2, 'db3' : db3, 'db4' : db4}
    
    return grads

In [217]:
def update_parameters(parameters, v, grads, learning_rate, t, beta1 = 0.9):
    
    L = len(parameters)//2 
    v_corrected = {}
    s_corrected = {}
    
    
    for l in range(L):
        v["dW" + str(l+1)] = beta1 * v["dW" + str(l+1)] + (1 - beta1) *  grads['dW' + str(l+1)]
        v["db" + str(l+1)] = beta1 * v["db" + str(l+1)] + (1 - beta1) *  grads['db' + str(l+1)]
        
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - (beta1**t))
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - (beta1**t))
        
        #s["dW" + str(l+1)] = beta2 * s["dW" + str(l+1)] + (1 - beta2) *  ((grads['dW' + str(l+1)])**2)
        #s["db" + str(l+1)] = beta2 * s["db" + str(l+1)] + (1 - beta2) *  ((grads['db' + str(l+1)])**2)
        
        #s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)]/(1- (beta2**t))
        #s_corrected["db" + str(l+1)] = s["db" + str(l+1)]/(1- (beta2**t))
        
        #/ (np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon)
        
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * v_corrected["dW" + str(l+1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * v_corrected["db" + str(l+1)])
            
    return parameters, v

In [220]:
def model(X, Y, layer_dimensions, learning_rate, iterations, lambd, beta1 = 0.9):
    
    np.random.seed(42)
    costs = []
    t = 0
    
    parameters = init_parameters(layer_dimensions)
    v = init_adam(parameters)
    
    for i in range(iterations):
        A4, cache = forward_propagation(X, parameters)
        cost = compute_cost(Y, A4, parameters, lambd)
        grads = backward_propagation(X, Y, cache, parameters, lambd)
        #parameters = update_parameters(parameters, grads, learning_rate)    'gd'
        #parameters, v = update_parameters(parameters, v, grads, learning_rate, beta)
        t = t + 1
        parameters, v = update_parameters(parameters, v,grads, learning_rate,t, beta1 = 0.9)
        
        if i % 100 == 0:
            print(f'Iteration {i}: {cost}')
         
        costs.append(cost)
         
    plt.plot(costs)
    
    return parameters

In [ ]:
%%time
para = model(x_train, y_train, layer_dim, 0.01, 2000, lambd = 0)

Iteration 0: 0.7098377523266454
Iteration 100: 0.6896045409349301
Iteration 200: 0.6866725819031566
Iteration 300: 0.6844395924314765
Iteration 400: 0.6824601761032733
Iteration 500: 0.6805724534465484


In [222]:
a4, _ = forward_propagation(x_train, para)
a4_dev, _ = forward_propagation(x_dev, para)

In [223]:
def predict(A4, Y, parameters):

    m = A4.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1,m))
    

    for i in range(0, A4.shape[1]):
        if A4[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    print("Accuracy: "  + str(np.sum((p == Y)/m)))
        
    return p

In [224]:
preditction_train = predict(a4, y_train, para)
prediction_test = predict(a4_dev, y_dev, para)

Accuracy: 0.5968
Accuracy: 0.49284525790349415


In [190]:
lr = LogisticRegression()
lr.fit(x_train.T, y_train.T)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [191]:
preds = lr.predict(x_dev.T)
from sklearn.metrics import confusion_matrix, classification_report

In [192]:
print(confusion_matrix(y_dev.T, preds))
print(classification_report(y_dev.T, preds))

[[668 801]
 [671 865]]
              precision    recall  f1-score   support

           0       0.50      0.45      0.48      1469
           1       0.52      0.56      0.54      1536

    accuracy                           0.51      3005
   macro avg       0.51      0.51      0.51      3005
weighted avg       0.51      0.51      0.51      3005

